# Sora2 Watermark Remover (Google Colab版)

このノートブックは、Sora動画からウォーターマークを除去するツールのGoogle Colab版です。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. セットアップ

### GitHubからリポジトリをクローン

In [ ]:
# GitHubリポジトリのクローン（毎回クリーン）
REPO_URL="https://github.com/fulfulggg/Sora2WatermarkRemover.git"
BRANCH="main"
WORKDIR="/content/Sora2WatermarkRemover"

%cd /content
!rm -f /content/remwm.py
!rm -rf "$WORKDIR"
!git clone --depth=1 -b "$BRANCH" "$REPO_URL" "$WORKDIR"
%cd "$WORKDIR"
!git rev-parse HEAD

print("\n✅ GitHubから最新を取得しました。")

### ⚠️ 重要: Pillow依存関係の修正

**🔴 以下のセルを実行すると自動的にランタイムが再起動されます**

再起動後は「2. 再起動後のセットアップ」から続行してください。

In [ ]:
# Pillow依存関係の修正（手動再起動のお願い）
import os
FLAG = '/content/pillow_restart_completed.flag'

if not os.path.exists(FLAG):
    get_ipython().run_line_magic('pip', 'uninstall -y pillow PIL')
    get_ipython().run_line_magic('pip', 'install -U "pillow==10.4.0"')
    get_ipython().run_line_magic('pip', 'install -U "websockets==15.0.1"')

    with open(FLAG, 'w') as f:
        f.write('done')

    print("\n" + "="*60)
    print("🔴 手動でランタイムを再起動してください")
    print("="*60)
    print("\n手順:")
    print("1. メニュー『ランタイム』→『ランタイムを再起動』")
    print("2. 再起動後『2. 再起動後のセットアップ』から続行")
    print("\n" + "="*60)
else:
    print("✅ すでに再起動済みです。次のセルへ進んでください。")

---

## 2. 再起動後のセットアップ

### ⬇️ ここから再開してください

In [ ]:
# Path動作確認（再起動後の最初のセル）
from pathlib import Path
import os

# 再起動フラグの確認
restart_flag = '/content/pillow_restart_completed.flag'
if os.path.exists(restart_flag):
    print("✅ ランタイム再起動済みを確認")
else:
    print("⚠️ まだランタイム再起動されていません。上のセルを実行してください。")

# Path動作テスト
print("Path動作テスト:", Path("/content").exists())  # True ならOK

if Path("/content").exists():
    print("✅ Pathlib正常動作を確認")
else:
    print("❌ エラー: ランタイムを再起動してください")

In [ ]:
# ディレクトリ移動（再起動後は必要）
%cd /content/Sora2WatermarkRemover
print("✅ 作業ディレクトリに移動しました")

In [ ]:
# Pillow確認
import PIL
import PIL._util as u

print(f"Pillow version: {PIL.__version__}")  # => 10.4.0
print(f"is_directory exists: {hasattr(u, 'is_directory')}")  # => True

if PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"):
    print("✅ Pillow依存関係は正常です。次のセルに進んでください。")
else:
    print("⚠️ 問題があります。ランタイムを再起動しましたか？")

In [ ]:
# 必要なパッケージのインストール（%pip 使用・バージョン固定）
get_ipython().run_line_magic('pip', 'install -U huggingface_hub==0.24.6 diffusers==0.26.3 websockets==15.0.1')
get_ipython().run_line_magic('pip', 'install --no-cache-dir --quiet transformers opencv-python tqdm loguru iopaint')
!apt-get -qq install -y ffmpeg

# HF Hub 互換シム（cached_download が無い版への対応）
try:
    import huggingface_hub as hfh
    from huggingface_hub import hf_hub_download as _hf
    if not hasattr(hfh, "cached_download"):
        hfh.cached_download = _hf
except Exception:
    pass

# LaMa モデルの事前ダウンロード（Python経由）
print("LaMaモデルをダウンロード中...")
try:
    from iopaint.download import cli_download_model
    cli_download_model("lama")
except Exception as e:
    print("iopaint.download 経由でのDLに失敗しました:", e)
    print("フォールバック: iopaint CLI を試行します…")
    !iopaint download --model lama

print("\n✅ パッケージ＋LaMaモデルの準備が完了しました。")

## 3. デバイス確認

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPUが利用できません。ランタイム→ランタイムのタイプを変更→GPUを選択してください。")

## 4. 動画アップロード

In [ ]:
from google.colab import files
import os, shutil

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

# 絶対パスに正規化し、/content 配下に固定（空白や(1)付き名称でも安全）
input_video = list(uploaded.keys())[0]
abs_src = os.path.abspath(input_video)
root, ext = os.path.splitext(abs_src)
input_path = f"/content/input_uploaded{ext or '.mp4'}"
if abs_src != input_path:
    shutil.move(abs_src, input_path)
print(f"✔ Uploaded → {input_path} (exists={os.path.exists(input_path)})")
UPLOAD_READY = True

## 5. 動画処理

### パラメータ設定

- `max_bbox_percent`: バウンディングボックスの最大サイズ（画像に対する割合%）
- `frame_step`: フレーム処理間隔（1=全フレーム、2=隔フレーム...）
- `target_fps`: 出力fps（0=入力と同じ）

In [ ]:
# パラメータ設定（必要に応じて調整）
max_bbox_percent = 10.0  # バウンディングボックスの最大サイズ
frame_step = 1           # 1=全フレーム、2=隔フレーム
target_fps = 0.0         # 0=入力と同じ

output_video = "output_no_watermark.mp4"

# ディレクトリ移動
%cd /content/Sora2WatermarkRemover

# 入力ファイルの確認（絶対パスを使用）
assert 'UPLOAD_READY' in globals() and UPLOAD_READY, "先にアップロードのセルを実行してください"
import os
output_path = f"/content/{output_video}"

assert os.path.exists(input_path), f"入力ファイルが見つかりません: {input_path}"
print(f"✅ 入力ファイル確認: {input_path}")
print(f"📝 出力先: {output_path}")
print(f"⚙️ パラメータ: max_bbox_percent={max_bbox_percent}, frame_step={frame_step}, target_fps={target_fps}")
print("\n🚀 処理を開始します...\n")

# remwm.pyを実行（GitHubから取得した絶対パスで呼び出し）
!python /content/Sora2WatermarkRemover/remwm.py "{input_path}" "{output_path}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps} \
  --overwrite

# 出力ファイルの確認
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
    print(f"\n✅ 処理完了: {output_path}")
    print(f"📊 ファイルサイズ: {file_size:.2f} MB")
else:
    print(f"\n❌ エラー: 出力ファイルが作成されませんでした")
    print("上記のエラーメッセージを確認してください")

## 6. 結果のダウンロード

In [ ]:
# 結果のダウンロード
print("結果をダウンロード:")
files.download(f"/content/{output_video}")

## 7. 使い方のヒント

### 処理速度を上げたい場合

```python
frame_step = 2  # 隔フレーム処理（約2倍速）
```

### 出力fpsを固定したい場合

```python
target_fps = 30  # 30fpsで出力
```

### 組み合わせ例

```python
frame_step = 2
target_fps = 30
# → 隔フレーム処理して30fpsで出力
```

## 8. トラブルシューティング

### ⚠️ pathlibエラー（AttributeError: 'PosixPath' object has no attribute '_str'）

**解決方法:**
- このノートブックは自動でランタイム再起動するため、通常は発生しません
- もし発生した場合は「ランタイム」→「ランタイムを再起動」を手動で実行

### その他のよくある問題

1. **pipインストールがハングアップする場合**
   - ランタイムを再起動してもう一度実行

2. **LaMaモデルが読み込めない場合**
   - `!iopaint download --model lama`を再実行
   - ランタイムを再起動

3. **GPUが使用できない場合**
   - ランタイム→ランタイムのタイプを変更→GPU（T4推奨）を選択

4. **メモリエラーの場合**
   - `frame_step`を大きくして処理フレーム数を減らす
   - 動画を短く分割して処理

### デバッグ用テストコード

```python
# Path動作確認
from pathlib import Path
print(Path("/content").exists())  # True ならOK
```